In [ ]:
import cv2
import mediapipe as mp
import pygame
import time
import math
import numpy as np

# =============================================================================
# Variables Globales para selección y modos de visualización
# =============================================================================
selected_points = []     # 4 puntos para la transformación en perspectiva
static_keys = False      # Fijar posiciones de teclas verdes
frontal_view = False     # Muestra la vista frontal (transformada)
rotate_180 = False       # Rota el frame 180 grados
mirror_effect = False    # Aplica efecto espejo

# Variables para octavas
current_octave = 4       
min_octave = 1
max_octave = 7
num_octavas = 1          # Modo: 1 (12 teclas) o 2 (24 teclas)
notes_in_octave = ["C", "Db", "D", "Eb", "E", "F", "Gb", "G", "Ab", "A", "Bb", "B"]
octave_sounds = []       

# =============================================================================
# Parámetros de estilo para botones compactos
# =============================================================================
button_height = 30
button_font_scale = 0.5
button_thickness = 1

buttons = {
    "reset":    {"pos": (10, 10),  "size": (60, button_height), "bg": (50, 50, 50), "text": "Rst"},
    "toggle":   {"pos": (80, 10),  "size": (70, button_height), "bg": (50, 50, 50), "text": "Togl"},
    "frontal":  {"pos": (160, 10), "size": (70, button_height), "bg": (50, 50, 50), "text": "Frnt"},
    "oct_down": {"pos": (240, 10), "size": (60, button_height), "bg": (50, 50, 50), "text": "Oct-"},
    "oct_up":   {"pos": (310, 10), "size": (60, button_height), "bg": (50, 50, 50), "text": "Oct+"},
}
text_color = (255, 255, 255)

# =============================================================================
# Funciones Auxiliares
# =============================================================================
def draw_button(frame, button_key, active_text=None):
    btn = buttons[button_key]
    x, y = btn["pos"]
    w, h = btn["size"]
    text = active_text if active_text is not None else btn["text"]
    cv2.rectangle(frame, (x, y), (x + w, y + h), btn["bg"], -1)
    (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, button_font_scale, button_thickness)
    text_x = x + (w - tw) // 2
    text_y = y + (h + th) // 2
    cv2.putText(frame, text, (text_x, text_y),
                cv2.FONT_HERSHEY_SIMPLEX, button_font_scale, text_color, button_thickness, cv2.LINE_AA)

def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def distance(point1, point2):
    return math.hypot(point1[0] - point2[0], point1[1] - point2[1])

# =============================================================================
# Callback del Mouse
# =============================================================================
def click_event(event, x, y, flags, param):
    global selected_points, static_keys, frontal_view, current_octave, num_octavas
    if event == cv2.EVENT_LBUTTONDOWN:
        # Comprueba si se hizo click en un botón
        for key, btn in buttons.items():
            bx, by = btn["pos"]
            bw, bh = btn["size"]
            if bx <= x <= bx + bw and by <= y <= by + bh:
                if key == "reset":
                    selected_points = []
                    print("Selección de puntos reiniciada (Reset).")
                elif key == "toggle":
                    static_keys = not static_keys
                    print("Modo estático activado." if static_keys else "Modo dinámico activado.")
                elif key == "frontal":
                    frontal_view = not frontal_view
                    print("Vista frontal activada." if frontal_view else "Vista frontal desactivada.")
                elif key == "oct_down":
                    if current_octave > min_octave:
                        current_octave -= 1
                        update_octave_sounds()
                        print("Octava bajada a", current_octave)
                    else:
                        print("Octava mínima alcanzada.")
                elif key == "oct_up":
                    if (num_octavas == 1 and current_octave < max_octave) or (num_octavas == 2 and current_octave < max_octave - 1):
                        current_octave += 1
                        update_octave_sounds()
                        print("Octava(s) subida(s) a", current_octave, ("" if num_octavas == 1 else f" y {current_octave+1}"))
                    else:
                        print("Octava máxima alcanzada.")
                return  # Se detectó click en un botón; no se registra como punto
        if len(selected_points) < 4:
            selected_points.append((x, y))
            print("Punto agregado:", (x, y))

# =============================================================================
# Inicialización de Pygame y carga de sonidos
# =============================================================================
pygame.mixer.init()
def load_sound(path):
    try:
        return pygame.mixer.Sound(path)
    except Exception as e:
        print(f"Error al cargar {path}: {e}")
        return None  # Retorna None en vez de salir

def update_octave_sounds():
    global octave_sounds, current_octave, num_octavas, notes_detected, last_triggered

    # 1) Cargar sonidos en una lista temporal según num_octavas
    temp_sounds = []
    octs_a_cargar = [current_octave] if num_octavas == 1 else [current_octave, current_octave + 1]
    for octave in octs_a_cargar:
        for note in notes_in_octave:
            filename = f"Piano_Samples/{note}{octave}.mp3"
            sound_obj = load_sound(filename)
            if sound_obj is None:
                print(f"No se pudo cargar el archivo: {filename}. Se cancela el cambio de octava.")
                return  # Cancela el cambio si falta alguna muestra
            temp_sounds.append(sound_obj)
    octave_sounds = temp_sounds
    print("Sonidos cargados para", 
          f"1 octava: {current_octave}" if num_octavas == 1 else f"2 octavas: {current_octave} y {current_octave+1}")

    # 2) Remapear las notas en modo estático si las teclas están fijas
    if static_keys and notes_detected:
        sorted_items = sorted(notes_detected.items(), key=lambda item: item[1]["pos"][0])
        needed_notes = 12 * num_octavas
        if len(sorted_items) == needed_notes:
            new_notes_detected = {}
            for i, (old_name, data) in enumerate(sorted_items):
                oct_index = current_octave if num_octavas == 1 else (current_octave + i // 12)
                note_letter = notes_in_octave[i % 12]
                new_name = f"{note_letter}{oct_index}"
                data["sound"] = octave_sounds[i]
                new_notes_detected[new_name] = data
                last_triggered[new_name] = last_triggered.pop(old_name, 0)
            notes_detected.clear()
            notes_detected.update(new_notes_detected)
        else:
            print(f"No se realizó el remapeo: se detectaron {len(sorted_items)} teclas pero se requieren {needed_notes}.")
            print("Se reiniciarán las teclas fijas para evitar errores.")
            notes_detected.clear()
            last_triggered.clear()

# Inicialización de sonidos iniciales
update_octave_sounds()

notes_detected = {}
last_triggered = {}
NOTE_COOLDOWN = 0.5

# =============================================================================
# Inicialización de MediaPipe Hands y OpenCV
# =============================================================================
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=3,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

cv2.namedWindow("Paper Piano", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Paper Piano", 640, 480)
cv2.setMouseCallback("Paper Piano", click_event)

# =============================================================================
# Ventana para ajustar HSV y presets preconfigurados
# =============================================================================
def nothing(x):
    pass

cv2.namedWindow("HSV Adjust", cv2.WINDOW_AUTOSIZE)
cv2.createTrackbar("Hmin", "HSV Adjust", 50, 179, nothing)
cv2.createTrackbar("Smin", "HSV Adjust", 100, 255, nothing)
cv2.createTrackbar("Vmin", "HSV Adjust", 20, 255, nothing)
cv2.createTrackbar("Hmax", "HSV Adjust", 100, 179, nothing)
cv2.createTrackbar("Smax", "HSV Adjust", 255, 255, nothing)
cv2.createTrackbar("Vmax", "HSV Adjust", 150, 255, nothing)

# =============================================================================
# Presets de valores HSV para el color verde
# =============================================================================
HSV_presets = {
    "Verde Oscuro": {"Hmin": 85, "Smin": 132, "Vmin": 113, "Hmax": 99, "Smax": 255, "Vmax": 255},
    "Verde Medio":   {"Hmin": 35, "Smin": 80,  "Vmin": 50,  "Hmax": 85,  "Smax": 255, "Vmax": 255},
    "Verde Claro":   {"Hmin": 50, "Smin": 50,  "Vmin": 80,  "Hmax": 71,  "Smax": 82,  "Vmax": 255}
}

# =============================================================================
# Función para actualizar los trackbars con un preset HSV
# =============================================================================
def set_HSV_trackbars(preset):
    cv2.setTrackbarPos("Hmin", "HSV Adjust", preset["Hmin"])
    cv2.setTrackbarPos("Smin", "HSV Adjust", preset["Smin"])
    cv2.setTrackbarPos("Vmin", "HSV Adjust", preset["Vmin"])
    cv2.setTrackbarPos("Hmax", "HSV Adjust", preset["Hmax"])
    cv2.setTrackbarPos("Smax", "HSV Adjust", preset["Smax"])
    cv2.setTrackbarPos("Vmax", "HSV Adjust", preset["Vmax"])
    print("Valores HSV aplicados:", preset)

# =============================================================================
# Función para detectar teclas verdes
# =============================================================================
def detectar_teclas_verdes(frame_hsv, lower_green, upper_green):
    mask_green = cv2.inRange(frame_hsv, lower_green, upper_green)
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
    cv2.imshow("Mask Green", mask_green)
    contornos, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contornos

# =============================================================================
# Bucle Principal
# =============================================================================
while True:
    ret, frame = cap.read()
    if not ret:
        print("No se pudo leer el frame.")
        break

    # Dibujar botones y mostrar octava en el frame original
    for key in buttons:
        draw_button(frame, key)
    octave_text = f"Oct: {current_octave}" if num_octavas == 1 else f"Oct: {current_octave} y {current_octave+1}"
    cv2.putText(frame, octave_text, (buttons["oct_up"]["pos"][0] + buttons["oct_up"]["size"][0] + 10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1, cv2.LINE_AA)
    for pt in selected_points:
        cv2.circle(frame, pt, 4, (255, 0, 0), -1)

    # Aplicar transformación en perspectiva solo si se han seleccionado 4 puntos
    if len(selected_points) == 4:
        pts = np.array(selected_points, dtype="float32")
        ordered_pts = order_points(pts)
        widthA = np.linalg.norm(ordered_pts[2] - ordered_pts[3])
        widthB = np.linalg.norm(ordered_pts[1] - ordered_pts[0])
        maxWidth = int(max(widthA, widthB))
        heightA = np.linalg.norm(ordered_pts[1] - ordered_pts[2])
        heightB = np.linalg.norm(ordered_pts[0] - ordered_pts[3])
        maxHeight = int(max(heightA, heightB))
        dst = np.array([[0, 0],
                        [maxWidth - 1, 0],
                        [maxWidth - 1, maxHeight - 1],
                        [0, maxHeight - 1]], dtype="float32")
        M = cv2.getPerspectiveTransform(ordered_pts, dst)
        warped = cv2.warpPerspective(frame, M, (maxWidth, maxHeight))
        display_frame = warped if frontal_view else frame.copy()
    else:
        display_frame = frame.copy()

    # Recalcular dimensiones a partir de display_frame
    h_frame, w_frame = display_frame.shape[:2]

    # Aplicar efectos de rotación y espejo
    if rotate_180:
        display_frame = cv2.rotate(display_frame, cv2.ROTATE_180)
        h_frame, w_frame = display_frame.shape[:2]
    if mirror_effect:
        display_frame = cv2.flip(display_frame, 1)
        h_frame, w_frame = display_frame.shape[:2]

    # Actualizar valores de HSV desde trackbars
    lower_green = np.array([cv2.getTrackbarPos("Hmin", "HSV Adjust"),
                             cv2.getTrackbarPos("Smin", "HSV Adjust"),
                             cv2.getTrackbarPos("Vmin", "HSV Adjust")], dtype=np.uint8)
    upper_green = np.array([cv2.getTrackbarPos("Hmax", "HSV Adjust"),
                             cv2.getTrackbarPos("Smax", "HSV Adjust"),
                             cv2.getTrackbarPos("Vmax", "HSV Adjust")], dtype=np.uint8)

    # Convertir a HSV y detectar teclas si no se usan teclas fijas
    frame_hsv = cv2.cvtColor(display_frame, cv2.COLOR_BGR2HSV)
    if not static_keys:
        contornos_verdes = detectar_teclas_verdes(frame_hsv, lower_green, upper_green)
        teclas_detectadas = []
        for cnt in contornos_verdes:
            if (area := cv2.contourArea(cnt)) < 50:
                continue
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            teclas_detectadas.append(((int(x), int(y)), int(radius)))
        if len(teclas_detectadas) == 12 * num_octavas:
            teclas_ordenadas = sorted(teclas_detectadas, key=lambda t: t[0][0])
            notes_detected.clear()
            last_triggered.clear()
            for i, (center, radius) in enumerate(teclas_ordenadas):
                oct_index = current_octave if num_octavas == 1 else (current_octave + i // 12)
                note_letter = notes_in_octave[i % 12]
                note_name = f"{note_letter}{oct_index}"
                notes_detected[note_name] = {"pos": center, "radius": radius, "sound": octave_sounds[i]}
                last_triggered[note_name] = 0

    # Dibujar círculos y nombres de notas detectadas
    for note_name, note_data in notes_detected.items():
        cv2.circle(display_frame, note_data["pos"], note_data["radius"], (0, 255, 0), 2)
        cv2.putText(display_frame, note_name, (note_data["pos"][0]-15, note_data["pos"][1]+5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

    # Procesar manos con MediaPipe usando display_frame
    rgb_frame = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(display_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            for idx in [4, 8, 12, 16, 20]:
                finger = hand_landmarks.landmark[idx]
                finger_pos = (int(finger.x * w_frame), int(finger.y * h_frame))
                cv2.circle(display_frame, finger_pos, 4, (0, 255, 255), cv2.FILLED)
                for note_name, note_data in notes_detected.items():
                    if distance(finger_pos, note_data["pos"]) < note_data["radius"]:
                        cv2.circle(display_frame, note_data["pos"], note_data["radius"], (0, 255, 0), 3)
                        current_time = time.time()
                        # Inicializar la clave si no existe para evitar KeyError
                        if note_name not in last_triggered:
                            last_triggered[note_name] = 0
                        if current_time - last_triggered[note_name] > NOTE_COOLDOWN:
                            note_data["sound"].play()
                            last_triggered[note_name] = current_time

    cv2.imshow("Paper Piano", display_frame)
    key = cv2.waitKey(1) & 0xFF

    # Mapeo de teclas a acciones
    if key in [ord('r'), ord('t'), ord('f'), ord('g'), ord('m'),
               ord('p'), ord('o'), ord('n'), ord('h'), ord('j'), ord('k'), ord('q')]:
        if key == ord('r'):
            selected_points = []
            print("Selección de puntos reiniciada (tecla 'r').")
        elif key == ord('t'):
            static_keys = not static_keys
            print("Modo estático activado." if static_keys else "Modo dinámico activado.")
        elif key == ord('f'):
            frontal_view = not frontal_view
            print("Vista frontal activada." if frontal_view else "Vista frontal desactivada.")
        elif key == ord('g'):
            rotate_180 = not rotate_180
            print("Rotación 180° activada." if rotate_180 else "Rotación 180° desactivada.")
        elif key == ord('m'):
            mirror_effect = not mirror_effect
            print("Efecto espejo activado." if mirror_effect else "Efecto espejo desactivado.")
        elif key == ord('p'):
            if (num_octavas == 1 and current_octave < max_octave) or (num_octavas == 2 and current_octave < max_octave - 1):
                current_octave += 1
                update_octave_sounds()
                print("Octava(s) subida(s) a", current_octave, ("" if num_octavas == 1 else f" y {current_octave+1}"))
            else:
                print("Octava máxima alcanzada.")
        elif key == ord('o'):
            if current_octave > min_octave:
                current_octave -= 1
                update_octave_sounds()
                print("Octava bajada a", current_octave)
            else:
                print("Octava mínima alcanzada.")
        elif key == ord('n'):
            num_octavas = 2 if num_octavas == 1 else 1
            print("Modo de", num_octavas, "octava" + ("s activado." if num_octavas == 2 else " activado."))
            update_octave_sounds()
        elif key == ord('h'):
            set_HSV_trackbars(HSV_presets["Verde Oscuro"])
        elif key == ord('j'):
            set_HSV_trackbars(HSV_presets["Verde Medio"])
        elif key == ord('k'):
            set_HSV_trackbars(HSV_presets["Verde Claro"])
        elif key == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Sonidos cargados para 1 octava: 4
Valores HSV aplicados: {'Hmin': 85, 'Smin': 132, 'Vmin': 113, 'Hmax': 99, 'Smax': 255, 'Vmax': 255}
Valores HSV aplicados: {'Hmin': 35, 'Smin': 80, 'Vmin': 50, 'Hmax': 85, 'Smax': 255, 'Vmax': 255}
Valores HSV aplicados: {'Hmin': 50, 'Smin': 50, 'Vmin': 80, 'Hmax': 71, 'Smax': 82, 'Vmax': 255}
Valores HSV aplicados: {'Hmin': 35, 'Smin': 80, 'Vmin': 50, 'Hmax': 85, 'Smax': 255, 'Vmax': 255}


# Referencias

## Similar Projects
- BTifmmp. (2024). GitHub - BTifmmp/paper-piano: Paper Piano uses Python and OpenCV to detect key presses on a hand-drawn piano, translating them into digital notes and sound. GitHub. https://github.com/BTifmmp/paper-piano
- google-ai-edge. (2025, February 7). GitHub - google-ai-edge/mediapipe: Cross-platform, customizable ML solutions for live and streaming media. GitHub. https://github.com/google-ai-edge/mediapipe
- Reddit - Explora lo que quieras. (2025). Reddit.com. https://www.reddit.com/r/computervision/comments/1es14rt/i_made_piano_on_paper_using_python_opencv_and/?rdt=62406
- MustafaLotfi. (2022). GitHub - MustafaLotfi/Paper-Piano: Using this program, you can play or control music by hand gesture. Also you can play a Paper Piano. GitHub. https://github.com/MustafaLotfi/Paper-Piano
- Saltwash. (2015, October 22). Paper Piano using Python and OpenCV. Electric Soup. https://rdmilligan.wordpress.com/2015/10/22/paper-piano-using-python-and-opencv/
- Instructables. (2021, December 12). Webcam Motion Piano With Python and OpenCV. Instructables. https://www.instructables.com/Webcam-Motion-Piano-With-Python-and-OpenCV2/
- (2025). Youtube.com. https://www.youtube.com/watch?v=F13tCcMjRLk&ab_channel=UmairSheikh

## Instrument Samples
- fuhton. (2017). GitHub - fuhton/piano-mp3: MP3 exports of piano keys. GitHub. https://github.com/fuhton/piano-mp3/tree/master
- nbrosowsky. (2018). GitHub - nbrosowsky/tonejs-instruments: A small instrument sample library with quick-loader for tone.js. GitHub. https://github.com/nbrosowsky/tonejs-instruments
- Freesound. (2025). Freesound.org. https://freesound.org/
- Free samples from Karoryfer. (2025). Karoryfer Samples. https://shop.karoryfer.com/pages/free-samples
- VSCO 2: Community Edition (CE) - Versilian Studios. (2024, September 30). Versilian Studios. https://versilian-studios.com/vsco-community/
- University of Iowa Electronic Music Studios. (2016). Uiowa.edu. https://theremin.music.uiowa.edu/

## Music Theory
- The ABC’s of the Musical Alphabet | Hub Guitar. (2018). Hubguitar.com. https://hubguitar.com/music-theory/the-musical-alphabet#:~:text=Memorize%20the%207%20letters%20of,A%E2%99%AF%2C%20B%2C%20C.